In [ ]:
!pip install zhon
!wget https://codeload.github.com/Concyclics/stopwords/zip/refs/heads/master 
!unzip master

import os
import jieba


In [ ]:
stopwords=[]

for dirname, _, filenames in os.walk('./stopwords-master'):
    for filename in filenames:
        with open(os.path.join(dirname, filename)) as f:
            if filename.endswith('txt'):
                stopwords+=list(f.read().split())
stopwords=list(set(stopwords))

In [ ]:
import re
from zhon.hanzi import punctuation as puncZH
from string import punctuation as puncEN

def hasPunctuation(s:str):
    f=False
    f|=bool(re.match(r"[%s]+" %puncZH, s))
    f|=bool(re.match(r"[%s]+" %puncEN, s))
    f|=bool(re.match(r"[\d\n\t\r]+", s))
    return f

In [ ]:
def getWords(news:str):
    split_news=jieba.lcut_for_search(news)
    removed_split_news=[]
    for text in split_news:
        if hasPunctuation(text) is False:
            removed_split_news.append(text)
    return removed_split_news

In [ ]:
def getWordCount(text:str):
    words = getWords(text)
    word_count={}
    for word in words:
        word_count[word]=word_count.get(word,0)+1
    for stopword in stopwords:
        if stopword in word_count:
            word_count.pop(stopword)
    return word_count

In [ ]:
text = '''
国家主席习近平7月28日晚应约同美国总统拜登通电话。两国元首就中美关系以及双方关心的问题进行了坦率沟通和交流。

习近平指出，当前，世界动荡和变革两种趋势持续演进，发展和安全两大赤字不断凸显。面对变乱交织的世界，国际社会和各国人民都期待中美两国发挥引领作用，维护世界和平安全，促进全球发展繁荣。这是中美两个大国职责所在。

习近平强调，从战略竞争的视角看待和定义中美关系，把中国视为最主要对手和最严峻的长期挑战，是对中美关系的误判和中国发展的误读，会对两国人民和国际社会产生误导。双方要保持各层级沟通，用好现有沟通渠道，推动双方合作。当前全球经济形势充满挑战。中美应该就宏观经济政策协调、维护全球产业链供应链稳定、保障全球能源和粮食安全等重大问题保持沟通。违背规律搞脱钩断链，无助于提振美国经济，也将使世界经济变得更加脆弱。双方要推动地区热点问题撤火降温，助力世界尽快摆脱新冠疫情，走出经济滞胀困局和衰退风险，维护以联合国为核心的国际体系和以国际法为基础的国际秩序。

习近平重点阐述了中方在台湾问题上的原则立场。习近平强调，台湾问题的历史经纬明明白白，两岸同属一个中国的事实和现状清清楚楚。中美三个联合公报是双方的政治承诺，一个中国原则是中美关系的政治基础。我们坚决反对“台独”分裂和外部势力干涉，绝不为任何形式的“台独”势力留下任何空间。中国政府和中国人民在台湾问题上的立场是一以贯之的，坚决维护中国国家主权和领土完整是14亿多中国人民的坚定意志。民意不可违，玩火必自焚。希望美方看清楚这一点。美方应该言行一致恪守一个中国原则，履行中美三个联合公报。

拜登表示，当今世界正处于一个关键时期。美中合作不仅有利于两国人民，也有利于各国人民。美方希望同中方保持畅通对话，增进相互了解，避免误解误判，寻求在利益交融的领域开展合作，同时妥善管控分歧。我愿重申，美国的一个中国政策没有改变也不会改变，美方不支持台湾“独立”。

两国元首还就乌克兰危机等交换了意见，习近平重申了中方原则立场。

两国元首认为，这次通话坦诚深入，同意保持联系，责成双方工作团队为此继续沟通合作。'''

In [ ]:
word_count = getWordCount(text)

In [ ]:
import pandas as pd
word_ds=pd.DataFrame(word_count,index=[0])
word_ds=word_ds.T
word_ds.columns=['count']

word_ds.sort_values(by=['count'],ascending=False,inplace=True)
word_ds.head(20)

In [ ]:
import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wc = WordCloud(font_path='../input/fonts-on-mac/Fonts/方正正中黑简体.TTF',width=1000,height=1000,background_color="red", max_words=50,color_func=lambda *args, **kwargs: (255, 255, 0))
wc.generate_from_frequencies(word_count)

wc.to_file("WordCloud.png")

plt.figure(figsize=(16,16))
plt.imshow(plt.imread("./WordCloud.png"))
plt.axis("off")
plt.show()